In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
from mu2e import mu2e_ext_path
from mu2e.dataframeprod import DataFrameMaker
from mu2e.mu2eplots import mu2e_plot, mu2e_plot3d
from matplotlib import gridspec
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
import math
import pandas as pd
from time import sleep
import plotly.io as pio
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from skhep.visual import MplPlotter as splt
from copy import deepcopy
from lmfit import Model, Parameters, report_fit
from hallprobesim_redux import *
from tqdm import tnrange, tqdm_notebook
#import pandas as pd
import seaborn as sns

<IPython.core.display.Javascript object>

In [2]:
init_notebook_mode(True)

In [3]:
from subprocess import call
import json
import plotly.utils
from plotly.offline import iplot, plot
from sys import getsizeof
import pickle as pkl

In [4]:
from mu2e.tools.fiteval import get_mag_field_function
import odespy
from mu2e.tools.particletransport import ElectronSwimmer

In [5]:
from scipy.interpolate import griddata

In [6]:
df_ds = DataFrameMaker(mu2e_ext_path+'datafiles/Mau13/Mu2e_DSMap_V13',input_type = 'pkl').data_frame

In [9]:
len(df_ds)

4902089

In [10]:
df_ds = df_ds.query("R<.700 and 5.000<Z<12.000")

In [11]:
len(df_ds)

685720

In [14]:
from sys import getsizeof

In [16]:
getsizeof(df_ds)/1000/1000

60.343384

In [17]:
df_ds.columns

Index(['X', 'Y', 'Z', 'Bx', 'By', 'Bz', 'R', 'Phi', 'Bphi', 'Br'], dtype='object')

In [20]:
df_ds = df_ds.drop(['R','Phi','Bphi','Br'],axis=1)

In [21]:
getsizeof(df_ds)/1000/1000

38.400344

In [22]:
x = 0.03
y = 0.03
zmin = 7.99
zmax = 8.03
#df_small = df_ds.query(f"{-x}<X<{x} and {-y}<Y<{y} and {zmin}<Z<{zmax}")
df_small = df_ds.query(f"-0.001<X<{x} and -0.001<Y<{y} and {zmin}<Z<{zmax}")

In [23]:
%timeit df_ds.query(f"-0.001<X<{x} and -0.001<Y<{y} and {zmin}<Z<{zmax}")

5.46 ms ± 69.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [83]:
def test():
    #cx = (df_ds.X < x) & (df_ds.X > -0.001)
    #cy = (df_ds.Y < y) & (df_ds.Y > -0.001)
    #cz = (df_ds.Z < zmax) & (df_ds.Z > zmin)
    #cc = cx & cy & cz
    #cc = (df_ds.X < x) & (df_ds.X > -0.001) & (df_ds.Y < y) & (df_ds.Y > -0.001) & (df_ds.Z < zmax) & (df_ds.Z > zmin)
    #return df_ds[cc]
    return df_ds[(df_ds.X < x) & (df_ds.X > -0.001) & (df_ds.Y < y) & (df_ds.Y > -0.001) & (df_ds.Z < zmax) & (df_ds.Z > zmin)]
#%timeit df_ds.query(f"-0.001<X<{x} and -0.001<Y<{y} and {zmin}<Z<{zmax}")

In [84]:
%timeit test()

4.7 ms ± 47 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
%timeit df_ds[pd.eval(f"(df_ds.X < {x}) & (df_ds.X > -0.001) & (df_ds.Y < {y}) & (df_ds.Y > -0.001) & (df_ds.Z < {zmax}) & (df_ds.Z > {zmin})")]

3.74 ms ± 35.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
x = 0.000
y = 0.000
z = 7.996
%timeit df_ds[pd.eval(f"(df_ds.X == {x}) & (df_ds.Y == {y}) & (df_ds.Z == {z})")]

2.72 ms ± 51 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
def interp()

In [24]:
len(df_small)

8

In [25]:
df_small

,X,Y,Z,Bx,By,Bz
2450981,0.000,0.000,7.996,-0.453110,-9.349171,10684.788088
2450982,0.000,0.000,8.021,-0.920105,-9.403955,10658.107507
2451502,0.000,0.025,7.996,-0.091459,4.426202,10684.328758
2451503,0.000,0.025,8.021,-0.566148,3.577890,10657.663800
2501518,0.025,0.000,7.996,13.244534,-9.126203,10683.930554
2501519,0.025,0.000,8.021,11.991061,-9.184468,10657.245990
2502039,0.025,0.025,7.996,13.590016,4.655356,10683.465529
2502040,0.025,0.025,8.021,12.328197,3.801989,10656.796063


In [49]:
%timeit grid_bx = griddata((df_small.X.values,df_small.Y.values,df_small.Z.values),df_small.Bx.values, (0.01,0.01,8.), method='linear')

245 µs ± 2.86 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [47]:
grid_bx

array(-0.52782889)

In [ ]:
#grid_bx = griddata((df_ds.X.values,df_ds.Y.values,df_ds.Z.values),df_ds.Bx.values, (0.01,0.01,8.), method='linear')

In [2]:
%timeit 1+1

5.71 ns ± 0.0694 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)
